In [4]:
from collections import defaultdict

from tqdm.auto import tqdm
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("../../data/itell-question-data-w-o3-scores.csv")
df

,id,response,score,condition,user_id,page_slug,chunk_slug,created_at,volume_slug,volume_title,page_title,chunk_header,chunk_text,question,answer,o3_mini_score
0,38f590ee-1964-4a7c-9625-53b53cb8a9f0,learning,0,NaN,clu91tykw0002l00fz3o1vfsh,emotional,01-Affect-Detection-from-Student-Activity-Data...,2024-04-15 19:57:38.656,cornell,Handbook of Learning Analytics,Emotional Learning Analytics,0.1 Affect Detection from Student Activity Data,Affective states cannot be directly measured b...,What approach was used to infer affect by anal...,"The interaction-based, log-file based, or sens...",1
1,95784937-f565-4450-bfad-babc2399d667,answer,0,NaN,cluc0hclp0000ju0flnuywkrr,emotional,01-Affect-Detection-from-Student-Activity-Data...,2024-05-05 20:29:28.14,cornell,Handbook of Learning Analytics,Emotional Learning Analytics,0.1 Affect Detection from Student Activity Data,Affective states cannot be directly measured b...,What approach was used to infer affect by anal...,"The interaction-based, log-file based, or sens...",1
2,88c0f433-7fdb-48ad-9898-a08d861635e1,human observers made live annotations regardin...,0,NaN,clu7cooq9000kjt0fbxh29vfy,emotional,01-Affect-Detection-from-Student-Activity-Data...,2024-04-15 19:03:37.985,cornell,Handbook of Learning Analytics,Emotional Learning Analytics,0.1 Affect Detection from Student Activity Data,Affective states cannot be directly measured b...,What approach was used to infer affect by anal...,"The interaction-based, log-file based, or sens...",2
3,53c99817-a318-45e5-a918-6b2e7dd4e8d1,Recorded data from student while doing help ba...,0,NaN,clucz38h40005jv0fhc5vjpfp,emotional,01-Affect-Detection-from-Student-Activity-Data...,2024-03-30 01:56:12.488,cornell,Handbook of Learning Analytics,Emotional Learning Analytics,0.1 Affect Detection from Student Activity Data,Affective states cannot be directly measured b...,What approach was used to infer affect by anal...,"The interaction-based, log-file based, or sens...",2
4,9f3eeb88-7254-402f-b025-b3c0298e2a64,analyzing context,0,NaN,clu8uwkw20000jz0f4yyg6c1d,emotional,01-Affect-Detection-from-Student-Activity-Data...,2024-05-03 20:56:52.298,cornell,Handbook of Learning Analytics,Emotional Learning Analytics,0.1 Affect Detection from Student Activity Data,Affective states cannot be directly measured b...,What approach was used to infer affect by anal...,"The interaction-based, log-file based, or sens...",2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1423,2257,"Slow technological progress, natural disasters...",2,stairs,pkwlvvzebu2ppqvjop66x5hequ,7-1-the-relatively-recent-arrival-of-economic-...,learning-objectives-700t,2024-08-15 22:19:30.781007+00:00,cttc-poe,Principles of Macroeconomics,7.1 The Relatively Recent Arrival of Economic ...,learning objectives,Let’s begin with a brief overview of spectacul...,What were economic conditions like before 1870?,Economic conditions before 1870 were marked by...,4
1424,1515,Economic conditions before 1870 were sluggish ...,2,stairs,mx25ylobcck2yg2xkiv4srpfh4,7-1-the-relatively-recent-arrival-of-economic-...,learning-objectives-700t,2024-08-12 18:03:45.775565+00:00,cttc-poe,Principles of Macroeconomics,7.1 The Relatively Recent Arrival of Economic ...,learning objectives,Let’s begin with a brief overview of spectacul...,What were economic conditions like before 1870?,Economic conditions before 1870 were marked by...,2
1425,2458,economic conditions were slow,2,stairs,kwj3wlbqwnfecwceaxe2noulju,7-1-the-relatively-recent-arrival-of-economic-...,learning-objectives-700t,2024-08-15 23:28:39.503359+00:00,cttc-poe,Principles of Macroeconomics,7.1 The Relatively Recent Arrival of Economic ...,learning objectives,Let’s begin with a brief overview of spectacul...,What were economic conditions like before 1870?,Economic conditions before 1870 were marked by...,2
1426,2794,"Before 1870, economic conditions were relative...",2,stairs,dggh43s5cosnmyxo57zex3cucy,7-1-the-relatively-recent-arrival-of-economic-...

In [75]:
def assign_questions_to_annotators(df, group_column='chunk_slug', responses_per_annotator=30):
    """
    Assign questions to annotators using a greedy bin packing approach.
    
    Args:
        df: DataFrame containing question responses
        group_column: Column name for the question identifier
        responses_per_annotator: Target number of responses per annotator
        
    Returns:
        DataFrame with an additional 'annotator_id' column
    """
    result_df = df.copy()
    result_df['annotator_id'] = None
    
    # Get questions and limit to responses_per_annotator
    limited_questions = {}
    for question_id, group in df.groupby(group_column):
        # Take only the first responses_per_annotator rows for each question
        limited_questions[question_id] = min(len(group), responses_per_annotator)
    
    # Map to track which responses have been assigned
    assigned_indices = defaultdict(set)  # question_id -> set of assigned indices
    
    # Track annotator assignments
    annotators = {}  # annotator_id -> [(question_id, num_responses), ...]
    current_annotator = 0
    
    for question_id, num_responses in limited_questions.items():
        # Get the first responses_per_annotator indices for this question
        question_indices = df[df[group_column] == question_id].index.tolist()[:responses_per_annotator]
        remaining_indices = [idx for idx in question_indices if idx not in assigned_indices[question_id]]
        
        actual_num_responses = len(remaining_indices)

        best_fit = None
        best_fit_space = responses_per_annotator + 1  # Initialize to larger than target
        
        for annotator_id, assigned in annotators.items():
            annotator_load = sum(resp for _, resp in assigned)
            available_space = responses_per_annotator - annotator_load
            
            # Find the annotator with the smallest sufficient space
            if actual_num_responses <= available_space and available_space < best_fit_space:
                best_fit = annotator_id
                best_fit_space = available_space
        
        if best_fit is not None:
            # Add question to best fitting annotator
            annotators[best_fit].append((question_id, actual_num_responses))
            
            # Update the result dataframe
            result_df.loc[remaining_indices, 'annotator_id'] = best_fit
            
            # Mark as assigned
            assigned_indices[question_id].update(remaining_indices)
        else:
            # Create new annotator for this question
            annotators[current_annotator] = [(question_id, actual_num_responses)]
            
            # Update the result dataframe
            result_df.loc[remaining_indices, 'annotator_id'] = current_annotator
            
            # Mark as assigned
            assigned_indices[question_id].update(remaining_indices)
            
            current_annotator += 1
    
    return result_df

df = assign_questions_to_annotators(df)
df

,id,response,score,condition,user_id,page_slug,chunk_slug,created_at,volume_slug,volume_title,page_title,chunk_header,chunk_text,question,answer,o3_mini_score,annotator_id
0,38f590ee-1964-4a7c-9625-53b53cb8a9f0,learning,0,NaN,clu91tykw0002l00fz3o1vfsh,emotional,01-Affect-Detection-from-Student-Activity-Data...,2024-04-15 19:57:38.656,cornell,Handbook of Learning Analytics,Emotional Learning Analytics,0.1 Affect Detection from Student Activity Data,Affective states cannot be directly measured b...,What approach was used to infer affect by anal...,"The interaction-based, log-file based, or sens...",1,0
1,95784937-f565-4450-bfad-babc2399d667,answer,0,NaN,cluc0hclp0000ju0flnuywkrr,emotional,01-Affect-Detection-from-Student-Activity-Data...,2024-05-05 20:29:28.14,cornell,Handbook of Learning Analytics,Emotional Learning Analytics,0.1 Affect Detection from Student Activity Data,Affective states cannot be directly measured b...,What approach was used to infer affect by anal...,"The interaction-based, log-file based, or sens...",1,0
2,88c0f433-7fdb-48ad-9898-a08d861635e1,human observers made live annotations regardin...,0,NaN,clu7cooq9000kjt0fbxh29vfy,emotional,01-Affect-Detection-from-Student-Activity-Data...,2024-04-15 19:03:37.985,cornell,Handbook of Learning Analytics,Emotional Learning Analytics,0.1 Affect Detection from Student Activity Data,Affective states cannot be directly measured b...,What approach was used to infer affect by anal...,"The interaction-based, log-file based, or sens...",2,0
3,53c99817-a318-45e5-a918-6b2e7dd4e8d1,Recorded data from student while doing help ba...,0,NaN,clucz38h40005jv0fhc5vjpfp,emotional,01-Affect-Detection-from-Student-Activity-Data...,2024-03-30 01:56:12.488,cornell,Handbook of Learning Analytics,Emotional Learning Analytics,0.1 Affect Detection from Student Activity Data,Affective states cannot be directly measured b...,What approach was used to infer affect by anal...,"The interaction-based, log-file based, or sens...",2,0
4,9f3eeb88-7254-402f-b025-b3c0298e2a64,analyzing context,0,NaN,clu8uwkw20000jz0f4yyg6c1d,emotional,01-Affect-Detection-from-Student-Activity-Data...,2024-05-03 20:56:52.298,cornell,Handbook of Learning Analytics,Emotional Learning Analytics,0.1 Affect Detection from Student Activity Data,Affective states cannot be directly measured b...,What approach was used to infer affect by anal...,"The interaction-based, log-file based, or sens...",2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1423,2257,"Slow technological progress, natural disasters...",2,stairs,pkwlvvzebu2ppqvjop66x5hequ,7-1-the-relatively-recent-arrival-of-economic-...,learning-objectives-700t,2024-08-15 22:19:30.781007+00:00,cttc-poe,Principles of Macroeconomics,7.1 The Relatively Recent Arrival of Economic ...,learning objectives,Let’s begin with a brief overview of spectacul...,What were economic conditions like before 1870?,Economic conditions before 1870 were marked by...,4,None
1424,1515,Economic conditions before 1870 were sluggish ...,2,stairs,mx25ylobcck2yg2xkiv4srpfh4,7-1-the-relatively-recent-arrival-of-economic-...,learning-objectives-700t,2024-08-12 18:03:45.775565+00:00,cttc-poe,Principles of Macroeconomics,7.1 The Relatively Recent Arrival of Economic ...,learning objectives,Let’s begin with a brief overview of spectacul...,What were economic conditions like before 1870?,Economic conditions before 1870 were marked by...,2,None
1425,2458,economic conditions were slow,2,stairs,kwj3wlbqwnfecwceaxe2noulju,7-1-the-relatively-recent-arrival-of-economic-...,learning-objectives-700t,2024-08-15 23:28:39.503359+00:00,cttc-poe,Principles of Macroeconomics,7.1 The Relatively Recent Arrival of Economic ...,learning objectives,Let’s begin with a brief overview of spectacul...,What were economic conditions like before 1870?,Economic conditions before 1870 were marked by...,2,None
1426,2794,"Before 1870, economic conditions were relative...",2,stairs,dggh43s5cosnmyxo57zex3cucy,7-1-the

In [92]:
# Get 12 random annotators
# grouped = list(df.sample(frac=1, random_state=42).groupby("annotator_id"))
# df = pd.concat(g[1] for g in grouped[:12])

In [93]:
print(df.volume_slug.value_counts())
df.groupby(["annotator_id", "chunk_slug"]).size()

volume_slug
cornell                                   681
cttc-poe                                  486
eloquent-javascript                       206
business-law-and-the-legal-environment     55
Name: count, dtype: int64


annotator_id  chunk_slug                                         
0             01-Affect-Detection-from-Student-Activity-Data-480t    30
1             02-Affect-Detection-from-Bodily-Signals-481t            6
              1-Emerging-Themes-482t                                  9
              2-Future-Themes-487t                                    9
              Abstract-Data-Types-818t                                2
                                                                     ..
38            header-148t                                             1
39            What-For,-Whom,-Why,-How?-Chunk-2-531t                 12
              introduction-322t                                       2
              while-and-do-Loops-789t                                 1
40            learning-objectives-700t                               30
Length: 184, dtype: int64

In [95]:
out_df = (
    df[~df.annotator_id.isna()]
    [[
        "annotator_id", "id", "chunk_header", "chunk_text", "question", "answer", "response"
    ]]
    .rename(columns={
        "id": "question_id",
        "answer": "reference_answer",
    })
    .assign(score="")
)
display(out_df.sample(3))
out_df.to_csv("../../data/itell-cri-data-for-annotators.csv", index=False)

,annotator_id,question_id,chunk_header,chunk_text,question,reference_answer,response,score
329,10,31bf81d0-00f1-49e7-93ea-d96ad7b9c436,Challenges Facing Learning Analytics About SRL,As software systems gathering trace data evolv...,How do self-regulating learners adapt their sk...,Self-regulating learners strive to adapt skill...,They utilize feedback,
995,31,1040,Rule of Law and Economic Growth,Economic growth depends on many factors. Key a...,What factors does economic growth depend on ac...,Adherence to the rule of law and protection of...,1. Slavery-based societies\n2. Global trading ...,
437,13,e85ea490-92fc-43bc-9c26-e3976bc037ba,Core Themes 3,One limitation of the face-based affect detect...,What is the advantage of combining interaction...,They can be applied to almost all cases while ...,They can be applied to almost all cases while ...,
